<a href="https://colab.research.google.com/github/beenleliu/Paddle/blob/master/chapter_ready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install mxnet

     |████████████████████████████████| 54.7MB 76kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [3]:
from mxnet import nd

MXNet **数据创建**

In [7]:
#创建一个行向量
x = nd.arange(12)
x
x.shape #实例x的形状
x.size #实例元素的总数
X = x.reshape(3,4) #形状改变为3行4列
X  


[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]
<NDArray 3x4 @cpu(0)>

In [13]:
#各元素全为0，形状为（2，3，4）的张量
nd.zeros((2,3,4))
#各元素全为1
nd.ones((2,3,4))
#利用list赋值
li = [[2,1,4,3],[1,2,3,4],[4,3,2,1]]
Y = nd.array(li)
Y


[[2. 1. 4. 3.]
 [1. 2. 3. 4.]
 [4. 3. 2. 1.]]
<NDArray 3x4 @cpu(0)>

In [14]:
#创建一个形状为（3，4），每个元素都随机采样于均值0，标准差为1的正态分布
nd.random.normal(0, 1, shape=(3,4))


[[ 2.2122064   0.7740038   1.0434403   1.1839255 ]
 [ 1.8917114  -1.2347414  -1.771029   -0.45138445]
 [ 0.57938355 -1.856082   -1.9768796  -0.20801921]]
<NDArray 3x4 @cpu(0)>

**数据操作**

In [49]:
#+
X+Y
#*
X*Y
#/
X/Y
#exp()：指数运算
Y.exp()
#dot函数,矩阵乘法，点乘
nd.dot(X,Y.T)
#矩阵连接
nd.concat(X,Y,dim=0) #dim=0,形状中的最左边元素连接
nd.concat(X,Y,dim=1) #dim=1,形状中左起第二个元素
X==Y #判断元素是否相同
X.sum() #元素和
X.norm().asscalar() #将结果变为Python的标量
X > Y
X < Y


[[1. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
<NDArray 3x4 @cpu(0)>

**广播机制**

In [33]:
A = nd.arange(3).reshape((3,1))
B = nd.arange(2).reshape((1,2))
A,B

(
 [[0.]
  [1.]
  [2.]]
 <NDArray 3x1 @cpu(0)>, 
 [[0. 1.]]
 <NDArray 1x2 @cpu(0)>)

In [34]:
#A中的第一列元素被广播到第二列,B中的第一行两个元素被广播到第二行和第三行
A+B 


[[0. 1.]
 [1. 2.]
 [2. 3.]]
<NDArray 3x2 @cpu(0)>

**索引**

In [35]:
X[1:3]


[[ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]
<NDArray 2x4 @cpu(0)>

In [37]:
X[1,2] = 9
X


[[ 0.  1.  2.  3.]
 [ 4.  5.  9.  7.]
 [ 8.  9. 10. 11.]]
<NDArray 3x4 @cpu(0)>

In [39]:
#给行索引为1的每一列元素赋值为12
X[1:2, :] = 12
X


[[ 0.  1.  2.  3.]
 [12. 12. 12. 12.]
 [ 8.  9. 10. 11.]]
<NDArray 3x4 @cpu(0)>

**NDArray和Numpy相互转换**

In [40]:
import numpy as np

In [47]:
#Numpy -> NDArray
P = np.ones((2,3))
D = nd.array(P)
#NDArray -> Numpy
D.asnumpy()

array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

**自动求梯度**

In [50]:
from mxnet import autograd, nd

对函数 y = 2x⊤x 求关于列向量 x 的梯度

In [51]:
#创建变量x,赋值
x = nd.arange(4).reshape((4,1))
x


[[0.]
 [1.]
 [2.]
 [3.]]
<NDArray 4x1 @cpu(0)>

In [52]:
#申请存储梯度所需要的内存
x.attach_grad()

In [56]:
#为了减少计算和内存开销，默认条件下MXNet不会记录⽤于求梯度的计算。我们需要调⽤record函数来要求MXNet记录与求梯度有关的计算。
#建立y函数
with autograd.record():
  y = 2 * nd.dot(x.T, x)


[[28.]]
<NDArray 1x1 @cpu(0)>

In [57]:
#调用backward函数自动求梯度
y.backward()

In [60]:
#函数 y = 2x⊤x 关于x的梯度应为4x
assert(x.grad - 4*x).norm().asscalar() == 0
x.grad


[[ 0.]
 [ 4.]
 [ 8.]
 [12.]]
<NDArray 4x1 @cpu(0)>

**训练模式和预测模式**

In [63]:
#默认情况下autograd还会将运⾏模式从预测模式转为训练模式。这可以通过调⽤is_training函数来查看
print(autograd.is_training())
with autograd.record():
  print(autograd.is_training())

False
True


**对Python控制流求梯度**

In [64]:
#即使函数的计算图包含了Python的控制流（如条件和循环控制），我们也有可能对变量求梯度。
def f(a):
  b = a * 2
  while b.norm().asscalar() < 1000:
    b = b * 2
  if b.sum().asscalar() > 0:
    c = b
  else:
    c = 100 * b
  return c

In [66]:
#使⽤record函数记录计算，并调⽤backward函数求梯度
# a = nd.random.normal(shape=1)
# a.attach_grad()
# with autograd.record():
#   c = f(a)
# c.backward()

In [69]:
a = nd.random.normal(12, shape=(3,4))
a.attach_grad()
with autograd.record():
  c = f(a)
c.backward()

In [70]:
a.grad == c/a


[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
<NDArray 3x4 @cpu(0)>